<a href="https://colab.research.google.com/github/jennif8r/LllForIFPR/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução
Bert só aceita 512 tokens, então para ele ler um arquivo e responder tem que ser com menos de 512 tokens.

In [1]:
!pip install transformers torch
!pip install huggingface_hub


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
from huggingface_hub import notebook_login

notebook_login()


In [10]:
import nltk

# Baixar o recurso 'punkt'
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
# Importar bibliotecas
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Carregar o tokenizer e o modelo BERT para QA
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Função para responder perguntas
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True, max_length=512, padding='max_length')
    input_ids = inputs['input_ids'].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    if answer_start >= answer_end:
        return ""  # Retorna uma string vazia se a resposta não for válida

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    return answer.strip()

# Função para ler o documento
def read_document(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Função para dividir contextos longos
def split_context(context, max_length=512):
    sentences = context.split('. ')
    split_contexts = []
    current_context = []

    for sentence in sentences:
        if len(tokenizer.tokenize(" ".join(current_context + [sentence]))) <= max_length:
            current_context.append(sentence)
        else:
            split_contexts.append(" ".join(current_context))
            current_context = [sentence]

    if current_context:
        split_contexts.append(" ".join(current_context))

    return split_contexts

# Caminho para o documento
document_path = '/content/test'

# Ler o conteúdo do documento
context = read_document(document_path)

# Dividir o contexto em partes menores se for muito longo
split_contexts = split_context(context)

# Pergunta de exemplo
question = "Quais cursos de graduação o IFPR oferece?"

# Obter a resposta para cada parte do contexto e escolher a melhor resposta
best_answer = ""
for part in split_contexts:
    answer = answer_question(question, part)
    if answer and answer != '[CLS]':  # Ajuste para ignorar respostas vazias
        best_answer = answer
        break

print(best_answer)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tecnico em administracao , tecnico em alimentos , tecnico em informatica , tecnico em administracao , tecnologia em alimentos , tecnologia em analise e desenvolvimento de sistemas , tecnologia em processos gerenciais
